<a href="https://colab.research.google.com/github/anshul-raman/stock_price_prediction/blob/master/Data_mining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import os
import sys
import bs4
import csv 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,GRU,Dense,Conv2D,MaxPool2D,Input,Dropout,Activation,Bidirectional,Concatenate,Lambda,Reshape,Conv1D,MaxPooling1D,LSTM
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import numpy as np

# ***QUANT DATA***

In [ ]:

GOOG_QUANT_DATA_FILE= pd.read_csv('/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/price/Quant/GOOG.csv')
GOOG_QUANT_DATA_FILE_LIST=(list(GOOG_QUANT_DATA_FILE['Date']))



Selecting and normalizing

In [ ]:
GOOG_QUANT_DATA_FILE=GOOG_QUANT_DATA_FILE[['Date','Open','High','Close','Low']]
print(GOOG_QUANT_DATA_FILE)
def normalizer(col):
  scaler=MinMaxScaler(feature_range=(0,1))
  df1=scaler.fit_transform(np.array(col).reshape(-1,1))
  return df1

GOOG_QUANT_DATA_FILE['Open']= normalizer(GOOG_QUANT_DATA_FILE['Open'])
GOOG_QUANT_DATA_FILE['Close']= normalizer(GOOG_QUANT_DATA_FILE['Close'])
GOOG_QUANT_DATA_FILE['High']= normalizer(GOOG_QUANT_DATA_FILE['High'])
GOOG_QUANT_DATA_FILE['Low']= normalizer(GOOG_QUANT_DATA_FILE['Low'])
loc_start_date= 0
for i in range(0,len(GOOG_QUANT_DATA_FILE_LIST)):
  if GOOG_QUANT_DATA_FILE_LIST[i]>=START_DATE:
    loc_start_date=i
    break




print(GOOG_QUANT_DATA_FILE_LIST[loc_start_date],loc_start_date)

   


           Date      Open      High     Close       Low
0    2013-07-01  0.052651  0.051619  0.057546  0.060324
1    2013-07-02  0.057934  0.050099  0.049725  0.049227
2    2013-07-03  0.043426  0.047523  0.055510  0.050993
3    2013-07-05  0.057666  0.056307  0.065424  0.063684
4    2013-07-08  0.070625  0.071594  0.081713  0.077666
..          ...       ...       ...       ...       ...
627  2015-12-24  0.923512  0.919095  0.920503  0.941553
628  2015-12-28  0.933041  0.951988  0.960280  0.949911
629  2015-12-29  0.971978  1.000000  1.000000  0.998645
630  2015-12-30  1.000000  0.993274  0.984213  1.000000
631  2015-12-31  0.979924  0.970385  0.950047  0.975330

[632 rows x 5 columns]
2014-01-02 128


# RAW NEWS DATA ***COLLECTION***

In [ ]:


GOOG_DATA_dir = os.listdir('/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG')
GOOG_DATA_dir=sorted(GOOG_DATA_dir)
print((GOOG_DATA_dir))

['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-11', '2014-01-12', '2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-19', '2014-01-20', '2014-01-21', '2014-01-22', '2014-01-23', '2014-01-24', '2014-01-25', '2014-01-26', '2014-01-27', '2014-01-28', '2014-01-29', '2014-01-30', '2014-01-31', '2014-02-01', '2014-02-02', '2014-02-03', '2014-02-04', '2014-02-05', '2014-02-06', '2014-02-07', '2014-02-08', '2014-02-09', '2014-02-10', '2014-02-11', '2014-02-12', '2014-02-13', '2014-02-14', '2014-02-15', '2014-02-16', '2014-02-17', '2014-02-18', '2014-02-19', '2014-02-20', '2014-02-21', '2014-02-22', '2014-02-23', '2014-02-24', '2014-02-25', '2014-02-26', '2014-02-27', '2014-02-28', '2014-03-01', '2014-03-02', '2014-03-03', '2014-03-04', '2014-03-05', '2014-03-06', '2014-03-07', '2014-03-08', '2014-03-09', '2014-03-10', '2014-03-11', '2014-03-12', '2014-03-13', '2014-03-14', '2014

In [ ]:
START_DATE=GOOG_DATA_dir[0]
END_DATE=GOOG_DATA_dir[-1]
print(START_DATE,END_DATE)

2014-01-01 2015-12-31


In [ ]:
GOOG_DATA_files = ['/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/' + x for x in GOOG_QUANT_DATA_FILE_LIST]
GOOG_DATA_files_1 = ['/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/' + x for x in GOOG_DATA_dir]

In [ ]:
def part_preprocess(fileName):
  sentences=[]
  if fileName in GOOG_DATA_files_1:
    f = open(fileName,'r').readlines()
  else :
    return sentences
  print(fileName)
  
  for i in f:
    i=json.loads(i)
    sentences.append(i['text'])
  Output = {} 
 
  for lis in sentences: 
      Output.setdefault(tuple(lis), list()).append(1) 


  for a, b in Output.items(): 
      Output[a] = sum(b)  
  sentences=list(Output.keys())    
  return sentences 


# ***Preprocessing***

In [ ]:
feature_word_data_GOOG=[]

for num in range(0,len(GOOG_DATA_files)):
  feature_word_data_GOOG.append(part_preprocess(GOOG_DATA_files[num]))

print(feature_word_data_GOOG)


/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-02
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-03
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-06
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-07
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-08
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-09
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-10
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-13
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-14
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/

In [ ]:
# l =max(feature_word_data_GOOG,key=len)
# for i in l:
#   print(i)

In [ ]:

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:

stops = set(stopwords.words("english"))     
            
print(stops)
def remove_stops(row):
    my_list = row

    meaningful_words = [w for w in my_list if not w in stops]

    return (meaningful_words)
def preprocess_tweet(news):
    processed_tweet = []
    tweet=news
    if True:
       
        new_text = []
        for word in tweet:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        tweet = " ".join(new_text)
 
    # Convert to lower case
    tweet = tweet.lower()
    tweet=  re.sub(r"\W", " ",tweet)
    tweet=  re.sub(r"\d", " ",tweet)
    tweet=  re.sub(r"-", " ",tweet)   
    tweet=  re.sub(r"_", " ",tweet)  
   
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'-(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    tweet = re.sub(r'\$', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    words = tweet.split()
    # print(tweet)


    return words




{'been', 'below', "it's", "mustn't", 'each', 'once', 'my', 'to', 'm', 'nor', 'our', 'isn', 'have', 'than', 'won', 'myself', 'do', 'from', 'herself', "mightn't", 'but', 'while', "shouldn't", "you'd", 'd', 'why', "you'll", 'will', 'these', "wasn't", 'you', "she's", 'can', 'itself', 'in', "wouldn't", 'same', "needn't", 'him', 'where', 're', 'does', 'having', 'be', 'before', 'those', 'how', 'we', 'being', 'has', 'themselves', 'weren', "don't", 'which', 'wasn', 'are', 'during', 'here', 'own', 'the', 'after', 'ours', 'ain', "didn't", "you've", 'such', 'that', "hasn't", 'both', "won't", 'and', 'an', 'not', 'only', 'couldn', "hadn't", 'too', 'who', 'doesn', "you're", 'they', "weren't", 'aren', 'up', 'wouldn', 'should', 'out', "shan't", 'down', 'all', 'more', 'were', "that'll", 'then', "aren't", 'ourselves', 'was', 'there', 'her', 'as', 'off', 'into', 'until', 'haven', 'a', 'other', 'hadn', 'between', 'she', 'll', 'didn', 'doing', 'i', "isn't", 'at', 'what', 'most', 'am', 'or', 'don', 'is', 'wi

In [ ]:
pre_features_GOOG=[]
for day in feature_word_data_GOOG:
  temp=[]
  for news in day:
    news=preprocess_tweet(news)
    news=remove_stops(news)
    
    temp=temp+news
  pre_features_GOOG.append(temp) 
 



In [ ]:
print((pre_features_GOOG[128]))
# l= min(pre_features_GOOG,key=len)
# print(len(l))
# print(l)
# l=0
# for i in pre_features_GOOG:
#   print(len(i))

['goog', 'google', 'inc', 'patent', 'issued', 'adding', 'new', 'attributes', 'structured', 'presentation', 'url', 'trade', 'signals', 'generated', 'twitter', 'stream', 'win', 'rate', 'still', 'open', 'goog', 'bidu', 'yelp', 'url', 'goog', 'apple', 'unlock', 'value', 'emulating', 'google', 'way', 'url', 'stock', 'stocks', 'stockaction']


# ***WORD TOKENIZATION***

In [ ]:
word_tokenizer = Tokenizer(oov_token="<OOV>")
word_tokenizer.fit_on_texts(pre_features_GOOG)
word_index = word_tokenizer.word_index
print(len(word_index))
print(word_index)
train_word_features_GOOG = word_tokenizer.texts_to_sequences(pre_features_GOOG)
padded_word_train_features_GOOG = pad_sequences(train_word_features_GOOG ,padding='post',maxlen=300,truncating="post")
print(padded_word_train_features_GOOG.shape)

7575
{'<OOV>': 1, 'goog': 2, 'url': 3, 'user': 4, 'google': 5, 'aapl': 6, 'fb': 7, 'stock': 8, 'stocks': 9, 'googl': 10, 'amzn': 11, 'inc': 12, 'twtr': 13, 'tsla': 14, 'msft': 15, 'nflx': 16, 'qqq': 17, 'new': 18, 'apple': 19, 'p': 20, 'research': 21, 'buy': 22, 'market': 23, 'news': 24, 'c': 25, 'pcln': 26, 'gpro': 27, 'like': 28, 'spy': 29, 'nasdaq': 30, 'x': 31, 'today': 32, 'tech': 33, 'get': 34, 'investing': 35, 'baba': 36, 'bidu': 37, 'via': 38, 'trading': 39, 'alphabet': 40, 'see': 41, 'stockaction': 42, 'sell': 43, 'hft': 44, 'watch': 45, 'video': 46, 'earnings': 47, 'next': 48, 'algos': 49, 'b': 50, 'sigma': 51, 'crossfinder': 52, 'android': 53, 'facebook': 54, 'could': 55, 'time': 56, 'yhoo': 57, 'price': 58, 'great': 59, 'us': 60, 'update': 61, 'ibm': 62, 'ats': 63, 'lx': 64, 'big': 65, 'mobile': 66, 'last': 67, 'k': 68, 'looking': 69, 'penny': 70, 'options': 71, 'lnkd': 72, 'free': 73, 'company': 74, 'high': 75, 'quant': 76, 'billion': 77, 'invest': 78, 'twitter': 79, 'shar

# ***CHAR TOKENIZATION***

In [ ]:
def char_featurizer(pos_feature_dis):
  char_feature = []
  for i in range(len(pos_feature_dis)):
    for j in range(min(len(pos_feature_dis[i]),300)):
      char_feature.append(list(pos_feature_dis[i][j]))
      
    
      
    if len(pos_feature_dis[i]) < 300 :
      if(len(pos_feature_dis[i])<=300):
        for j in range(300-len(pos_feature_dis[i])):
          char_feature.append([])
     

  return (char_feature)

pre_char_features_GOOG= char_featurizer(pre_features_GOOG)



In [ ]:
char_tokenizer = Tokenizer(oov_token="<OOV>")

char_tokenizer.fit_on_texts(pre_char_features_GOOG)

char_index = char_tokenizer.word_index
CHAR_MAX_LENGTH =15

train_charword_sequences = char_tokenizer.texts_to_sequences(pre_char_features_GOOG)
padded_charword_features_GOOG = pad_sequences(train_charword_sequences,padding='post',maxlen=CHAR_MAX_LENGTH,truncating="post")
print(padded_charword_features_GOOG.shape)

padded_charword_features_GOOG= padded_charword_features_GOOG.reshape(-1,300,CHAR_MAX_LENGTH)
print(padded_charword_features_GOOG.shape)

(189600, 15)
(632, 300, 15)


# ***CREATING QUANT DATA SETS***

In [ ]:
def create_dataset_quant(pos_start,dataset, time_step=1,output_time_step=14):
	dataX, dataY = [], []
	for i in range(pos_start,len(dataset)-output_time_step):
		a = dataset[(i-time_step):i]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i:i + output_time_step][:])
	return np.array(dataX), np.array(dataY)


#important params
input_time_step=100 
output_time_step=30
QUANT_DATA= np.array(GOOG_QUANT_DATA_FILE[['Open','Close','High','Low']])
QUANT_DATA_INPUT ,QUANT_DATA_OUT=create_dataset_quant(loc_start_date,QUANT_DATA,input_time_step,output_time_step)

In [ ]:
print(QUANT_DATA_INPUT.shape,QUANT_DATA_OUT.shape)

(474, 100, 4) (474, 30, 4)


# ***CREATING NEWS DATA SET***

In [ ]:
news_input_chardata= padded_charword_features_GOOG[loc_start_date:-output_time_step]
news_input_worddata= padded_word_train_features_GOOG[loc_start_date:-output_time_step]


print(news_input_chardata.shape,news_input_worddata.shape)


(474, 300, 15) (474, 300)


# TRAIN TEST ***DATA***

# ***MODEL***

In [ ]:
from tensorflow.keras.layers import Flatten,GRUCell
from tensorflow.keras.layers import GRU,LSTM
import numpy as np
from tensorflow.keras.models import Model

from tensorflow.keras.layers import RepeatVector,concatenate,Dot,Lambda
from tensorflow.keras.layers import Reshape


In [ ]:
def base_model(input_time_step):
  
  input_shape = (input_time_step,4)

  input= Input(shape=input_shape,dtype='float32',name='input')
 
  X = (LSTM(units=64,return_sequences=True))(input)

  X = (LSTM(units=64,return_sequences=True))(X)
  print(X.shape)
  X = Dropout(rate=0.5)(X)
  print(X.shape)
  X = (LSTM(units=30))(X)
  # X= Reshape((-1,1))(X)
  # X = (LSTM(units=30,return_sequences=True))(X)
  # print(X.shape)
  X = Dropout(rate=0.2)(X)
  print(X.shape)
 

  X = Dense(units=4)(X)

  print(X.shape)

  model = Model(inputs=[input],outputs=X)
  return model

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.007)
model = base_model(input_time_step)
model.compile(optimizer=opt, loss='mean_squared_error') 

print(model.summary())

(None, 100, 64)
(None, 100, 64)
(None, 30)
(None, 4)
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 100, 4)]          0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 100, 64)           17664     
_________________________________________________________________
lstm_44 (LSTM)               (None, 100, 64)           33024     
_________________________________________________________________
dropout_20 (Dropout)         (None, 100, 64)           0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 30)                11400     
_________________________________________________________________
dropout_21 (Dropout)         (None, 30)                0         
_________________________________________________________________


In [ ]:

history=model.fit(QUANT_DATA_INPUT,QUANT_DATA_OUT,validation_split=0.2,epochs=10,batch_size=32)

ValueError: ignored

In [ ]:
model.predict(QUANT_DATA_INPUT)

array([[[0.25008395, 0.27734363, 0.242868  , 0.27673575],
        [0.32366675, 0.36277694, 0.32263175, 0.36590353],
        [0.33178747, 0.36403346, 0.33875397, 0.36849198],
        ...,
        [0.34975475, 0.36128706, 0.34881645, 0.3515972 ],
        [0.35543215, 0.3709826 , 0.3549033 , 0.3611957 ],
        [0.35590145, 0.37207395, 0.35384825, 0.3618126 ]],

       [[0.25010517, 0.2773741 , 0.24288616, 0.2767678 ],
        [0.32376814, 0.36291498, 0.32272115, 0.36604846],
        [0.3319838 , 0.364331  , 0.33892056, 0.36877865],
        ...,
        [0.34975702, 0.36128795, 0.34882087, 0.35160044],
        [0.35543138, 0.370979  , 0.35490283, 0.36119345],
        [0.35590422, 0.37207675, 0.35385194, 0.3618162 ]],

       [[0.2501225 , 0.27739888, 0.24290097, 0.27679393],
        [0.32385242, 0.36302966, 0.32279548, 0.3661689 ],
        [0.33215174, 0.36458513, 0.3390633 , 0.3690236 ],
        ...,
        [0.34975833, 0.36128783, 0.3488241 , 0.3516024 ],
        [0.35543054, 0.370975

array([[[0.37522265, 0.37384062, 0.36928643, 0.380838  ],
        [0.37457476, 0.36243802, 0.36813221, 0.37605751],
        [0.37177179, 0.37973851, 0.37084891, 0.37822515],
        ...,
        [0.46636973, 0.48205243, 0.47362251, 0.47264509],
        [0.47880715, 0.47715147, 0.47099015, 0.4858096 ],
        [0.46718675, 0.49570181, 0.48492603, 0.48334046]],

       [[0.37457476, 0.36243802, 0.36813221, 0.37605751],
        [0.37177179, 0.37973851, 0.37084891, 0.37822515],
        [0.38866021, 0.40998602, 0.40017051, 0.39935731],
        ...,
        [0.47880715, 0.47715147, 0.47099015, 0.4858096 ],
        [0.46718675, 0.49570181, 0.48492603, 0.48334046],
        [0.48811766, 0.49977413, 0.49124631, 0.50220436]],

       [[0.37177179, 0.37973851, 0.37084891, 0.37822515],
        [0.38866021, 0.40998602, 0.40017051, 0.39935731],
        [0.41823975, 0.41331418, 0.41091106, 0.41677125],
        ...,
        [0.46718675, 0.49570181, 0.48492603, 0.48334046],
        [0.48811766, 0.499774